Download Jupyter Notebook File: [ipynb File](/notebooks/2023-06-13-solar-land-area.ipynb)

How many times have you heard the claim that "we can power the world with solar photovoltaic (PV) power". [After encountering many false claims](/2023/06/13//false-claims-about-solar-potential),  I will now subject it to thorough engineering analysis. I will outline the critical assumptions needed to estimate the land area that solar farms would need to cover including un-panelled site-access area.

Several methodologies exist for estimating the land area capacity of solar power. I will utilize 'Average Practical PV Output' data, which is available country by country in terms of kWh/kWp/day. I.e. average energy output to grid per unit of capacity per day. This method, widely employed and supported by renewable energy advocates, conveniently provides a modest accounting for several confounding factors associated with solar energy production including panel efficiency, self-shading and soiling.

This study does not account for all the auxiliary areas associated with solar farms. These areas include extensive mining and manufacturing facilities necessary for the production and maintenance of solar panels.

The analysis adopts a 'copper-plate model'. This modeling approach overlooks the infrastructure and limitations of energy transport, assuming, simplistically, that all system components are situated in one place.

Land area calculations for solar capacity are based on preliminary site selection. However, as securing ideal sites becomes more challenging, later sites will likely be less optimal. Consequently, this study does not take into account the self-shading effect of the undulating terrain itself, or other siting limitations that can affect the efficiency of solar energy production.

This analysis does not consider the potential role of thermal solar power, which has the potential to outperform PV technology in providing efficient, low-grade water-heating solutions.

Primary energy consumption data is relatively consistent across sources such as BP, EIA, and IEA. For this analysis, I will use data from BP, the most frequently cited source for primary energy information.

### Capacity Factor Data

The capacity factor is the actual output of the panels divided by the maximum output potential of the panels for a whole year. This is obviously better in sunny countries and worse in less-sunlit countries. The capacity factor also changes dramatically between winter and summer in countries which are far from the equator, so here we are looking at a year-average.

The capacity factor for each country can be obtained from the [Global Photovoltaic Power Potential by Country](https://datacatalog.worldbank.org/search/dataset/0038379) dataset. This data, is compiled by [Solargis](https://solargis.com) and was funded by the World Bank.

This data operates under the assumption of freestanding structures utilizing monofacial crystalline silicon PV modules, which are mounted at an optimal tilt to maximize annual energy yield. It also considers the use of high-efficiency inverters.

The data factors in solar radiation, air temperature, and terrain to simulate the energy conversion and losses in the PV modules and other components of a PV power plant. It presumes a 3.5% loss due to dirt and soiling, while the aggregate impact of other conversion losses—including inter-row shading, mismatch, inverters, cables, and transformers—is estimated at 7.5%. The power plant availability is assumed to be 100%.

The key column we need, labeled 'Average practical potential kWh/kWp/day' in the source data essentially represents the year-averaged capacity factor multiplied by 24. [Refer to footnote for more details](#footnotes).

In [1]:
import pandas as pd
import numpy as np
from numpy import pi, sqrt, log
import sympy as sp
# from sympy import pi, sqrt, log
import requests
import folium
from handcalcs.decorator import handcalc
import handcalcs.render
from IPython.display import Markdown
from bs4 import BeautifulSoup
# from pint import UnitRegistry
# si = UnitRegistry()

import forallpeople as si
# try:
si.environment('energy_custom')
# except:
#     si.environment("default")
# pd.set_option("display.precision", 1)
import io

show = lambda func: display(Markdown(func))
hc = handcalc(jupyter_display=True)

In [2]:
# # Solargis: Solar data
# import os
# URL = 'https://datacatalogfiles.worldbank.org/ddh-published/0038379/DR0046831/solargis_pvpotential_countryranking_2020_data.xlsx'
# file = os.path.basename(URL)
# response = requests.get(URL)

# with open(file, 'wb') as f:
#     f.write(response.content)
    
# df =pd.read_excel('solargis_pvpotential_countryranking_2020_data.xlsx', sheet_name='Country indicators', header=[1])
# df.columns = (df.columns
#               .str.replace('\n',' ', regex=True)
#               .str.replace(' +',' ', regex=True))
# df = df.rename(columns={"Country or region":"Country"})
# df.to_csv('solargis.csv', index=False)


In [3]:
df =pd.read_csv('solargis.csv')


HOURS =24
solargis=pd.DataFrame()
solargis["capacity_factor"] = df['Average practical potential (PVOUT Level 1, kWh/kWp/day), long-term']/HOURS

METRES = 1E6 # METRES per Kilometre
solargis["country_area_m2"] = df['Total area, 2018']*METRES
solargis[["Country", "ISO_A3"]] = df[["Country", "ISO_A3"]]
solargis = solargis.dropna(how="any")

df = (
    solargis
.set_index("Country")[["capacity_factor"]]
.sort_values("capacity_factor", ascending=False)
.rename(columns={"capacity_factor":"Capacity Factor [%]"})
* 100
)

In [4]:
pd.set_option("display.precision", 0)
df

,Capacity Factor [%]
Country,
Namibia,22
Chile,22
Jordan,22
Arab Republic of Egypt,22
Republic of Yemen,22
...,...
Estonia,12
Isle of Man (U.K.),12
Norway,12


Some of the least sunlit regions correspond to the most developed countries worldwide, a fact that may surprise many. Equally surprising is that the sunniest regions only possess double the capacity factor of the least sunny ones. However, the capacity factor isn't the only determinant of potential solar energy utilization.

A significant challenge confronting these northern countries is the temporal disparity of solar energy availability. The majority of solar energy is received during long summer days, and currently, there are no efficient methods to store such large quantities of electricity for an extended period of six months.

The average global capacity factor, when accounting for land area, is as follows:

In [5]:
%%render 2
C_f_global = np.average(solargis["capacity_factor"], weights=solargis["country_area_m2"])


<IPython.core.display.Latex object>

### Solar Potential Assumptions

We can now begin to formulate a reasonable estimate for the net amount of electricity solar power can deliver, taking all lifetime costs into account. Essentially, we're seeking to determine the average annual solar power output per unit of land area, expressed in electrical energy output, per second, per square metre. This is expressed in the unit of "watts" per square meter (W·m⁻²). 

The most crucial factor is the installed capacity of existing solar farms. It may seem intuitive to calculate solar PV potential simply by multiplying the incident radiation energy by the panel's efficiency. However, this approach overlooks several important aspects.

We must also account for factors like construction, maintenance, disposal, recycling, and transportation, which are currently supported by the existing fossil fuel infrastructure. These elements play a significant role in the comprehensive understanding and evaluation of solar power potential. 

In [6]:
solar_assumptions = """
Symbol,Name,Description,Source
C_f,Capacity Factor,Average output divided by installed capacity (from SolarGIS paid for by World Bank),known per country
C,Installed Capacity [W/m^2], Installed capacity (max possible output) of solar power per unit land area in Watts / square metre,assumption
E,Embodied Energy, Total lifetime output minus that used to construct and maintain the panel as a proportion of lifetime output,assumption
A,Availability, The proportion of time that the plant is available to generate power
"""
solar_assumption_table = pd.read_csv(io.StringIO(solar_assumptions), index_col=0)
show(solar_assumption_table.to_markdown())

| Symbol   | Name                       | Description                                                                                                  | Source            |
|:---------|:---------------------------|:-------------------------------------------------------------------------------------------------------------|:------------------|
| C_f      | Capacity Factor            | Average output divided by installed capacity (from SolarGIS paid for by World Bank)                          | known per country |
| C        | Installed Capacity [W/m^2] | Installed capacity (max possible output) of solar power per unit land area in Watts / square metre           | assumption        |
| E        | Embodied Energy            | Total lifetime output minus that used to construct and maintain the panel as a proportion of lifetime output | assumption        |
| A        | Availability               | The proportion of time that the plant is available to generate power                                         | nan               |

$$Production\ [W / m^2]= C_f \cdot C \cdot E$$

As a first start, lets take the following optimistic estimates and apply them to the global totals to get a minimum estimate of the global land area we will need to cover with solar farms.

<h3 id="installed">Installed Capacity of Farms</h3>

What is call the installed capacity is often called the '[capacity density](https://www.cell.com/cms/10.1016/j.joule.2021.03.005/attachment/c2b79392-88f9-4231-833d-faf1a0a2a125/mmc1)', 'land-use efficiency' or LUE in the literature.

The installed capacity per unit area depends on what sort of panel efficiency and layout density the solar farm can achieve. The relevant value is the “MW-ac” – the alternating current that it is capable of sending to the Grid after electrical inversion. This is approximately 80% of the “MW-dc”. Hereafter, I refer to MW-ac simply as MW.

- [California Solar PV Land Use efficiency](https://www.researchgate.net/publication/259386034_Land-Use_Efficiency_of_Big_Solar) in 2013 was **35W/m2**.
- [The US Average  Land Use efficiency](https://www.nrel.gov/docs/fy13osti/56290.pdf) in 2013  was **42.5 W/m2** (5.8 acres/MW) for a Fixed Axis, Large (>20MW) capacity-weighted average.
- [](https://www.cell.com/cms/10.1016/j.joule.2021.03.005/attachment/c2b79392-88f9-4231-833d-faf1a0a2a125/mmc1)

In [7]:
%%render params 0
C = (50 * si.W / si.m**2)

<IPython.core.display.Latex object>

<h3 id="embodied">Embodied Energy</h3>

It typically takes the first [2 years](https://cat.org.uk/info-resources/free-information-service/energy/solar-photovoltaic/) of the solar panel's 20 year life to pay back the embodied energy consumed in the manufacturing. We have to account for this by discounting at least 10% of the lifetime energy output. The first two years are the most productive part of the panel life, but lets assume energy output doesn't decay through the lifetime. Some energy is also required for cleaning, repair, recycling, transport and disposal. 



In [8]:
%%render 2
E = 1- 0.1 - 0.01

<IPython.core.display.Latex object>

<h3 id="availability">Availability Factor</h3>

In a solar PV power plant, the plant availability factor is one of the important factors to be evaluated. This depends on the operative functioning of various components and grid regulation. This paper looked at a single industrial scale plant over 5 years, and found an average of [95%](https://www.sciencedirect.com/science/article/pii/S1876610218301917). From the analysis, it is observed that tripping time greatly effects the availability factor of inverter as well as the PV plant.

In [9]:
%%render

A = 0.95

<IPython.core.display.Latex object>

Needless to say, these estimates are unrealistic, especially the overbuild factor. But they provide a bare minimum unachievable according to the laws of physics. 

Now I will use these scaling factors to calculate the total area of solar panels required.

Therefore solar can produce:

In [10]:
%%render 1

Production = C_f_global  * C *E *A

<IPython.core.display.Latex object>

---
## Total Power Consumption

### Primary Energy consumption data

[BP Statistical Review of World Energy](https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy/downloads.html) is the most widely trusted and canonical estimate of primary energy consumption used worldwide.

The definition of Primary energy is the total chemical energy in the fossil fuels burned that year, plus the the equivalent amount of fossil fuel input required to generate electricity produced by non-fossil sources in a fossil fuel power plant assuming 40.625% efficiency. [See footnotes](#footnotes)

BP only lists the consumption of the 81 main countries. Other countries are categories in to groups such as "Other Caribbean". 

I immediately convert the data "primary_ej" into annual averaged Watts.


In [11]:
# # BP: Primary Energy
# URL  = 'https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/xlsx/energy-economics/statistical-review/bp-stats-review-2022-consolidated-dataset-panel-format.csv'
# file = os.path.basename(URL)
# response = requests.get(URL)
# with open(file, 'wb') as f:
#     f.write(response.content)

# df = pd.read_csv(file)

# df = df[df["Year"] == 2021][["Country","ISO3166_alpha3", "primary_ej"]]
# df.to_csv("bp.csv", index=False)


In [12]:
# Load the necessary slice of Primary Energy data from 2021
df = pd.read_csv("bp.csv")

# Convert to year-averaged Watts 
JOULES = 1E18 # JOULES per Exajoule
SECONDS =  3.154e+7 # SECONDS per year

df["primary_w"] = df["primary_ej"]*JOULES/SECONDS
bp = (
    df
    .rename(columns={"ISO3166_alpha3":"ISO_A3"})
    .dropna(how="any")
    .sort_values("primary_w", ascending=False)
)    

df = ((bp
 .drop(["ISO_A3", "primary_ej"], axis=1)
 .set_index("Country")
 .rename(columns={"primary_w":"Primary Energy Consumption [GW]"})
 [["Primary Energy Consumption [GW]"]]
 / 1e9)
 .round(0)
)

pd.set_option("display.precision", 0)
df

,Primary Energy Consumption [GW]
Country,
Total World,18870
Total Non-OECD,11581
Total Asia Pacific,8638
Total OECD,7289
China,4998
...,...
Other Southern Africa,6
Latvia,5
Luxembourg,5


In [13]:
%%render 0
P_global  = bp["primary_w"].iloc[0] * si.W

<IPython.core.display.Latex object>

But now, we need to make a few assumptions in order to use these values to calculate land area. This is where the common-sense comes in. 

## Energy Demand Assumptions

The first step is to scale the primary energy down according to some assumptions to get the actual energy demand we are intending to supply

In [14]:
energy_assumptions = """
Symbol,Name,Description,Source
P,Primary Energy [W],Average consumption in Watts averaged over most recent year: 2021 (from BP),known per country
G,Growth in Consumption,Factor by which the primary energy consumption has grown when this plan is implemented,assumption
S,Solar Proportion, Fraction of energy that solar will supply,assumption
R,Efficiency Factor, Factor by which electrifying transport and heating and changing behavior can reduce final energy demand,assumption
D,Electricity Correction, Factor by which to correct the total amount of primary energy ignoring waste heat from non-fossil fuels when producing electricity
O,Overbuild,Factor of overproduction required to account for insufficient storage for intermittent supply,assumption
"""
energy_assumption_table = pd.read_csv(io.StringIO(energy_assumptions), index_col=0)
show(energy_assumption_table.to_markdown())

| Symbol   | Name                   | Description                                                                                                                        | Source            |
|:---------|:-----------------------|:-----------------------------------------------------------------------------------------------------------------------------------|:------------------|
| P        | Primary Energy [W]     | Average consumption in Watts averaged over most recent year: 2021 (from BP)                                                        | known per country |
| G        | Growth in Consumption  | Factor by which the primary energy consumption has grown when this plan is implemented                                             | assumption        |
| S        | Solar Proportion       | Fraction of energy that solar will supply                                                                                          | assumption        |
| R        | Efficiency Factor      | Factor by which electrifying transport and heating and changing behavior can reduce final energy demand                            | assumption        |
| D        | Electricity Correction | Factor by which to correct the total amount of primary energy ignoring waste heat from non-fossil fuels when producing electricity | nan               |
| O        | Overbuild              | Factor of overproduction required to account for insufficient storage for intermittent supply                                      | assumption        |

$$Energy\ Demand\ [W]=  P_{global} \cdot D \cdot G \cdot S \cdot R \cdot O$$ 


<h3 id ="correction">Electrical Demand Correction</h3>

Note that these primary energy figures from BP (and IEA and EIA) were originally designed to measure fossil fuel consumption not the useful energy that they provide accounting for conversion efficiency. Obviously, the primary energy used in generating electricity is much higher than the electricity provided as most of the chemical energy of the fuel is lost as waste heat, out of the cooling towers. So when it comes to non-fossil generated electricity, to make for a useful comparison, they convert the electricity generated by non-fossil fuels, in terms of the fossil fuel energy that would have been required to produce the  was generated using a conversion efficiency of 40.625%. 

In other words the electrical component of this of this fossil fuel demand is only 40% of the primary energy figures in the BP data. About 1/3 of primary energy is used to generate electricity. Therefore, nuclear and renewables only need to replace 1/3 * 0.4 + 2/3 = 80% of primary energy supply to meet demand.

In [15]:
%%render 1
D = 0.8

<IPython.core.display.Latex object>

### Proportion of Solar

Lets assume that solar supplies all our power as many people smart people seem to believe. For example Elon Musk is very bullish on the potential of solar. At [an on-stage conference appearance](https://youtu.be/4caLiMgsub4), he says “you could easily power all of China with solar”, to rapturous applause. Twitter is often swayed by infographics claiming the same. For example: an area of panels the size of [Spain accompanied by the claim](https://twitter.com/waitbutwhy/status/1397698410380251136?s=20), that it could power the world. Elon replied with: ‘Exactly’. Elon Musk used a graphic in his Tesla Power Wall presentation in 2015 in which he showed a tiny blue square, the size of Connecticut intended to represent the area of solar panels to power the entire USA.

[This excellent video](https://youtu.be/IZEaYjo4ZJU?t=1002) claims that the area required to power all of the world is close to the size of france (551,695 square km), 


In [16]:
%%render 
S = 1

<IPython.core.display.Latex object>

<h3 id="growth">Growth in Consumption</h3>

Lets make the unrealistic assumption that growth remains constant despite the [certainty](https://ourworldindata.org/worlds-energy-problem) that it will rise. The OECD countries (Europe,  North America, Australia and East Asia) represent around half the world's energy consumption despite possessing less than 1/5 of the population. But the developing world is catching up fast. For the Non-OECD to just to reach the standard of OECD, we will have to triple global energy consumption. Lets assume, miraculously, that it remains constant.

<iframe src="https://ourworldindata.org/grapher/global-primary-energy" loading="lazy" style="width: 100%; height: 600px; border: 0px none;"></iframe>

In [17]:
%%render params
G = 1 


<IPython.core.display.Latex object>

<h3 id="efficiency">Electrical Efficiency Factor</h3>

Electrifying heating and transport can make things more efficient. It important to distinguish this from the fact that most of the fossil fuel energy used to generate electricity is waste heat which we don't need to provide at all - we've already accounted for this;

[I delved into the electrification efficiency question separately](https://sebbeck.com/2023/06/20/how-much-energy-will-electrification-save). In summary, according to the most optimistic observers, we can hope to [save 40%](https://www.sciencedirect.com/science/article/pii/S2542435117300120#sec2.4) from:

>1. The higher energy-to-work conversion efficiency of using electricity for heating, heat pumps, and electric motors, and using electrolytic hydrogen in hydrogen fuel cells for transportation, compared with using fossil fuels (Table S4);
>2. The elimination of energy needed to mine, transport, and refine coal, oil, gas, biofuels, bioenergy, and uranium;
>3. Assumed modest additional policy-driven energy-efficiency measures beyond those under BAU.
>
>These factors decrease average demand ∼23.0%, 12.6%, and 6.9%, respectively, for a total of 42.5%. Thus, WWS not only replaces fossil-fuel electricity directly but is also an energy-efficiency measure, reducing demand.

The mental gymnastics involved in getting to this number are great. Notably, the figure of 40% savings ignores the fact that electrifying end-use demand incurs a cost of 4-15% of grid-losses. There are other biases which interfere with this estimate. Therefore, I will estimate 20% savings.


In [18]:
%%render 1
R = 0.8

<IPython.core.display.Latex object>

<h3 id="overbuild">Overbuild Factor</h3>

[Capacity credit](https://www.osti.gov/pages/biblio/1479875), is how much of nameplate capacity can counted on as to reliably serve load at peak demand. Solar without storage has a capacity value of 0 when the peak demand is close to sunset, which it normally is, which makes solar functionally useless without storage.

The key problem is storage. The terms "primary," "secondary," and "tertiary" are often used to refer to different types of reserve or response services. These are services that [actually exist](https://www.nationalgrideso.com/document/88586/download#:~:text=Secondary%20response%20is%20an%20automatic,primary%20response%20requirement%20is%20higher.):

1. Primary Reserve/Response on a second to second basis is regualted by spinning reserves from traditional power plants (i.e., additional capacity from generators that are already connected to the system and spinning but not producing power) 
2. Secondary Reserve/Response: within 30 seconds, automatic increase in generation (or reduction of demand) when the frequency is below 50Hz.
3. Tertiary typically involves bringing new plants online, within an hour or so.

But no country has (with the exception of high-hydro penetration countries such as Norway and Brazil) have enough storage to make it through a single night. [I have done calculations](https://sebbeck.com/2023/07/08/false-claims-about-solar-potential) for nightly electricity storage requirements. These are unfeasible in terms of cost (4000x global GDP) for the lithium, lithium reserves (24x times speculated reserves) and lithium production rate (17,000 years to mine the sufficient lithium). We can forget about inter-day storage.

Beyond this is seasonal storage. We need seasonal storage in order to not freeze to death in the winter. The United States for example [consumes 52%](https://www.eia.gov/totalenergy/data/monthly/) of its primary energy during the colder half of the year (October to March), but the solar output the US in this period is only [44% of the total.](https://datacatalog.worldbank.org/dataset/global-photovoltaic-power-potential-country)

This is a big issue because earlier, we assumed we were only going to build enough solar panels so that our solar energy production matched the energy consumption over the course of the whole year, not each month.

At least 6% (1763TWh) of annual energy demand would need to be stored from the summer to be used over the winter (again using Jacobson's optimistic 42.5% electrical efficiency factor). The brand-new battery storage facility in Escondido, California, has a capacity of 120MWh and uses 0.00486km2. We would need 14.7 million of these. In terms of land area this is 267x267km, 71,400km<sup>2</sup>, 0.9% of US lower 48, the size of Ireland.

The obvious solution is to this is to increase the solar capacity so that seasonal storage isn't required. The most adverse month December, with a capacity factor of 13% and a supply shortfall 667TWh of solar electricity. Covering this requires the addition of 25000km<sup>2</sup>, 158x158km of solar farmed land.

In fact, because of the difficulty of storing electrical energy, most sane plans actually propose an overbuild capacity of 5 to 15 times the annual demand. This is to account for the low capacity value of solar. Capacity value is how much of nameplate capacity can counted on as to reliably serve load at peak demand. For solar, this value is 0% during the colder half of the year because peak demand occurs during darkness.

A factor of 5 overbuild for capacity is 840000km2, 917x917km. This is the minimum extent of solar farms required to reduce energy storage to an overnight problem. Not much more can be said here.



In [19]:
%%render 0 
O = 5

<IPython.core.display.Latex object>

Thus, we end up with the final energy demand of:

In [20]:
%%render 1
EnergyDemand = P_global * D *G * S * R * O

<IPython.core.display.Latex object>

---

## Area of Land Required

To get the land area required, we just divide the energy consumption by energy production intensity of the land.

In [21]:
%%render 2
A_solar = EnergyDemand / Production 


<IPython.core.display.Latex object>

It seems like a lot. But what does it mean in terms of the space we have left on planet earth?

In [22]:
A_land = 129949283 * 1000000 * si.m**2 # global ice-free land area

In [23]:
%%render
A_land # ice free
PercentGlobalLandArea = A_solar / A_land * 100 # \% Minimum of Earth's land area required

<IPython.core.display.Latex object>

Therefore a minimum of around a percent of the earth's land area is required to sustain our present energy needs. 

[Land is primarily used for agriculture](https://ourworldindata.org/land-use). Note how on this diagram, only 1% is 'Urban area and built up land'. Its true that a huge quantity of land is managed for farming. But it is one thing tending to crops and another thing building physical infrastructure.

![](https://ourworldindata.org/uploads/2020/01/Global-land-use-graphic-800x511.png)

<iframe src="https://ourworldindata.org/grapher/land-use-over-the-long-term?time=600..latest&facet=none" loading="lazy" style="width: 100%; height: 600px; border: 0px none;"></iframe>

[A study](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3841857/) of global inventory of the spatial distribution and density of roads, parking lots, buildings, driveways, sidewalks and other manmade surfaces, which it called 'constructed impervious surface area (ISA)' was published in 2007 by a collaborative team involving US National Geophysical Data Center, NASA and US Department of Energy. It measured the reflectivity of the ground using satellite imagery. The global ISA represents the extant footprint of every structure human civilisation has created. It estimated the ISA to be 579,703 km². An area the size of France.

In [24]:
%%render 1
A_structures = 579_703 * 1_000_000 * si.m**2
AllHumanStructures = A_structures / A_land * 100 # \%


<IPython.core.display.Latex object>

The solar area is technically the land-use for the entire farm, of which around half to a third depending on access requirements and the need for spacing to overcome self-shading. So the necessary panel area is at a minimum, several times larger than that of existing human structures.

The point here is that when we are looking at the feasibility of installing physical infrastructure such as solar panels, the relevant point of comparison it is land occupied by other physical structures. Agricultural land, wild land or forest land is incomparable. So, [this typical article from 'nextenergysolar'](https://www.nextenergysolarfund.com/thought-leadership-white-papers/land-use/) spends thousands of words failing to mention the relevant point.

Lets calculate the area required for all nations using the same parameters:

In [25]:
# Merge all data together
latlon = pd.read_csv("country.csv")
isa = pd.read_csv("isa.csv") # from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3841857/table/t1-sensors-07-01962/?report=objectonly
forest = pd.read_csv("forest.csv") # from https://www.gapminder.org/data/ in 2020
agri = pd.read_csv("agri.csv") # from https://www.gapminder.org/data/ in 2020
rural = pd.read_csv("rural.csv") # from https://www.gapminder.org/data/ # in 2010
urban = pd.read_csv("urban.csv") # from https://www.gapminder.org/data/ # in 2010
installed = pd.read_csv("solar-installed.csv") # from https://en.wikipedia.org/wiki/Solar_power_by_country, via https://www.irena.org/Publications/2023/Mar/Renewable-capacity-statistics-2023 # in 2023

In [26]:
bp_solargis = (latlon
               .merge(solargis.drop("Country", axis=1), on='ISO_A3', how="left")
               .merge(bp.drop("Country", axis=1), on='ISO_A3', how="left")
               .merge(isa.drop("Country", axis=1), on='ISO_A3', how="left")
               .merge(forest.drop("country", axis=1), on='ISO_A3', how="left")
               .merge(agri.drop("country", axis=1), on='ISO_A3', how="left")
               .merge(rural.drop("country", axis=1), on='ISO_A3', how="left")
               .merge(urban.drop("country", axis=1), on='ISO_A3', how="left")
               .merge(installed.drop("country", axis=1), on='ISO_A3', how="left")
               
)
bp_solargis.to_json("bp_solargis.json", orient="records")



In [30]:
# calculate the area for all countries
def getsolararea(P, C_f,S,R,C,O,G,E):
    A_solar = P * G * S*R *O / ( C_f *C * E) # m**2 
    return A_solar

def getsolarradius(**kwargs):
    A_solar = getsolararea(**kwargs)
    r_solar = sqrt(A_solar/pi)
    return r_solar

farms = bp_solargis.copy()
farms["farm_area_m2"] = getsolararea(P=farms["primary_w"], C_f=farms["capacity_factor"], S=S,R=R, C=C.value, O = O,G = G, E = E )
farms["farm_radius_m"] = getsolarradius(P=farms["primary_w"],C_f=farms["capacity_factor"], S=S,R=R, C=C.value, O = O,G = G, E = E )
farms["farm_fraction"] = farms["farm_area_m2"] / farms["country_area_m2"]
# farms.sort_values("farm_fraction", ascending=False)

pd.set_option("display.precision", 3)
pd.set_option('display.float_format', '{:.1f}'.format)
(farms[["Country","farm_fraction"]]
 .sort_values("farm_fraction", ascending=False)
 .set_index("Country")
 .dropna(how="any")
 .rename(columns={"farm_fraction":"Land Required by Solar Farms [%]"}) * 100
)

,Land Required by Solar Farms [%]
Country,
Singapore,9390.3
Hong Kong,1743.5
Netherlands,246.2
"Korea, Republic of",231.2
Qatar,230.9
...,...
Kazakhstan,2.0
Argentina,1.9
Algeria,1.4


It actually doesn't look so bad for many of the less densely populated, sunnier countries. The problem is in europe in particular, the place that seems most interested by solar power.

<div   style="position: relative; width: 100%; height: 0; padding-bottom: 100%">
    <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" src="/2023/07/04/solar-farm-map.html"></iframe>
</div>


In [ ]:
# import ipywidgets as widgets
# from IPython.display import display
# import folium

# # Define function that generates a map
# def generate_map(df):
#     m = folium.Map(location=[50, 5], zoom_start=5)
#     data = [tuple(x) for x in zip(df['lat'], df['lon'], df['farm_radius_m'], df["farm_fraction"])]
#     for lat, lon, radius, fraction in data:
#         label = f'Percent land: { fraction:,.0%}, Radius: {radius/1000:.0f} km'
#         folium.Circle(
#             location=[lat, lon],
#             radius=radius,
#             color="blue",
#             fill=True,
#             fill_color="blue",
#             popup=folium.Popup(label)
#         ).add_to(m)
#     display(m)


# generate_map(farms)

---

# Footnotes

### Solargis - Average practical potential

Average practical potential kWh<sub>output</sub> · day<sup>-1</sup> · kW<sup>-1</sup><sub>capacity</sub> is actually just capacity factor times 24.

For example, a solar farm with a  kWh<sub>output</sub> · day<sup>-1</sup> · kW<sup>-1</sup><sub>capacity</sub> means it produces 1 kWh per day, for each 1 kW of capacity

### Primary Energy

BP is the source for present and historic consumption used by [Our World in Data](https://ourworldindata.org/energy-production-consumption#how-much-energy-does-the-world-consume). 

[Gap-Minder](https://www.gapminder.org/data/) still quotes the World Bank estimates of primary energy even though this is no-longer available.

The definition of Primary energy is the:
- total chemical energy in the fossil fuels burned that year, 
- plus the the equivalent amount of fossil fuel input required  
to generate electricity produced by non-fossil sources in a fossil fuel power plant assuming 40.625% efficiency.
- plus an estimate of the biomass chemical energy burned from the supply of electricity using biomass assuming 32% efficiency.

> Traditionally, in bp’s Statistical Review of World Energy, the primary energy of non-fossil based electricity (nuclear, hydro, wind, solar, geothermal, biomass in power and other renewables sources) has been calculated on an ‘input-equivalent’ basis – i.e. based on the equivalent amount of fossil fuel input required to generate that amount of electricity in a standard thermal power plant.

For non-fossil electricity (excluding biomass-powered electricity)

> 2018 onwards: the annual rate of efficiency improvement is based on the simplified assumption that efficiency will increase linearly [from 40%] to 45% by 2050. [I.e. it is assumed as 40.625% in 2022]

For Bio-mass plants:

> In this edition, we assume a constant efficiency of 32% for biomass power to better reflect the actual efficiency of  biomass power plants.

In [ ]:
# ## slider-based interactive map

# df = farms.copy()
# df["unscaled_farm_radius_m"] = getsolarradius(P=bp_solargis["primary_w"],C_f=bp_solargis["capacity_factor"], S=1,R=1, C=1, O = 1,G = 1, E = 1, M = 1  )


# # Define function that generates a map


# def generate_scaled_map(df, R , O , S, G, C ,E, M):
#     scale_factor = getsolarradius(P=1, C_f=1, R=R , O=O , S=S, G=G, C=C ,E=E, M=M)
#     print(scale_factor)
#     df["farm_radius_m"] = scale_factor * df["unscaled_farm_radius_m"]
#     m = folium.Map(location=[50, 5], zoom_start=5)
#     data = [tuple(x) for x in zip(df['lat'], df['lon'], df['farm_radius_m'])]
#     for lat, lon, radius in data:
#         label = f'Radius: {radius}'
#         folium.Circle(
#             location=[lat, lon],
#             radius=radius,
#             color="blue",
#             fill=True,
#             fill_color="blue",
#             popup=folium.Popup(label)
#         ).add_to(m)
#     display(m)


# RANGE = 2
# # Create widgets
# electrification = widgets.FloatSlider(min=R/RANGE, max=R*RANGE, value=R, step=R*0.1, description='R, Electrification')
# overbuild = widgets.FloatSlider(min=O/RANGE, max=O*RANGE, value=O, step=O*0.1, description='O, Overbuild:')
# solar_proportion = widgets.FloatSlider(min=S/RANGE, max=S*RANGE, value=S, step=S*0.1, description='S, Solar Proportion:')
# growth = widgets.FloatSlider(min=G/RANGE, max=G*RANGE, value=G, step=G*0.1, description='G, Growth in demand:')
# installed_capacity = widgets.FloatSlider(min=C/RANGE, max=C*RANGE, value=C, step=C*0.1, description='C, Installed capacity:')
# embodied_energy = widgets.FloatSlider(min=E/RANGE, max=E*RANGE, value=E, step=E*0.1, description='E, Embodied:')
# maintenance_energy = widgets.FloatSlider(min=M/RANGE, max=M*RANGE, value=M, step=M*0.1, description='M, Maintenance:')

# # Use the interact function to automatically create the UI
# widget = widgets.interactive(generate_scaled_map, df=widgets.fixed(df), R=electrification , O=overbuild , S=solar_proportion, G=growth,C=installed_capacity ,E=embodied_energy, M=maintenance_energy)

# # Display the widget
# display(widget)